In [2]:
import pyterrier as pt
if not pt.started():
  print ("not")  
  pt.init()

not
PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [3]:
#!pip install --upgrade fastrank lightgbm 
#!pip install python-terrier

In [1]:
import pickle
from my_functions import extractorRoberta
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
my_extractor = extractorRoberta(my_device = device, model_path = '/notebook/cqas/external_pretrained_models/')
print ("loaded extractors")

/notebook/cqas/external_pretrained_models/roberta.hdf5
/notebook/cqas/external_pretrained_models/vocab_dir
encoder loaded
indexer loaded
model path  /notebook/cqas/external_pretrained_models/roberta.hdf5
model loaded
reader loaded
loaded extractors


In [2]:
def extract_objs_asp(model_for_extraction, input_string):
    model_for_extraction.from_string(input_string)
    obj1, obj2, predicates, aspects = model_for_extraction.get_params()
    return (obj1.lower(), obj2.lower(), predicates, aspects)


def count_score1(text, nlu_tuple):
    (obj1, obj2, pred, asp) = nlu_tuple
    r = 1.0
    if (len(obj1) != 0 and len(obj2) != 0):
        if (len(pred) != 0):
            pred = re.sub('[!#?,.:";]', '', pred[0])
            if (obj1 in text and obj2 in text and pred in text):
                r += 1.0
        if (len(asp) != 0):
            asp = re.sub('[!#?,.:";]', '', asp[0])
            if (obj1 in text and obj2 in text and asp in text):
                r += 1.0
        elif (obj1 in text and obj2 in text):
            r = 1.5
        elif (obj1 in text or obj2 in text):
            r = 1.2
    else:
        if (obj1) in text or (obj2) in text:
            r = 1.2
    return r

def count_score_nlu(nlu_tuple):
    if (len(nlu_tuple[0]) == 0):
        return 0.0
    else: return 1.0


def count_score(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    if (len(obj1) != 0 and len(obj2) != 0):
        if (obj1 in text):
            r += 1.0
        if (obj2 in text):
            r += 1.0
        for asp in asps:
            if asp in text:
                r += 1.5
        for pred in preds:
            if pred in text:
                r += 1.0
    else:
        if ((obj1) in text and len(obj1)!= 0) or (obj2 in text and len(obj2) != 0):
            r = 1.0
    return r

def count_score_obj(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    if (len(obj1) != 0 and obj1 in text):
        r += 1.0
    if (len(obj2) != 0 and obj2 in text):
        r += 1.0
    return r

def count_score_asp_pred(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    o1 = (len(obj1) != 0 and obj1 in text)
    o2 = (len(obj2) != 0 and obj2 in text)
    if (o1 or o2):
        for asp in asps:
            if asp in text:
                r += 0.5
        for pred in preds:
            if pred in text:
                r += 0.5
    return r


def make_scores_obj(query, answers):
    print ("make_scores_obj")
    (obj1, obj2, pred, asp) = extract_objs_asp(extr, query)
    print ("in make scores", obj1, obj2, pred, asp)
    scores_answers = [count_score(cleanhtml(answer), (obj1, obj2, pred, asp)) for answer in answers]
    return scores_answers

In [3]:
from xml.dom import minidom
import re


def read_xml(filename):
    # convert file filename to list of tuples (number_of_topic, title_of_topic) 
    # input: filename string
    # output: list of corresponding tuples
    answer_list = []
    xmldoc = minidom.parse(filename)
    itemlist = xmldoc.getElementsByTagName('topics')
    print(len(itemlist))
    print(itemlist)
    topic_list = itemlist[0].getElementsByTagName('topic')
    print (len(topic_list))
    for topic in topic_list:
        tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
        answer_list.append(tuple_for_add)
    return answer_list

topics_2020 = read_xml('topics-task-2.xml')

1
[<DOM Element: topics at 0x7fb6d48b7050>]
50


In [4]:
topics_2020[45]

('46', '\nWhich beverage has more calories per glass: beer or cider?\n')

In [5]:
import requests
#r = requests.post('http://10.30.99.211:8261/gpt_small', data = "What is better for deep learning Python or Matlab?")
#print (r.status_code)
import requests
from xml.dom import minidom
import sys
import argparse

#numpy
import numpy as np

Read retireved documents and qrels:

In [6]:
import pickle


with open('/notebook/touche/list_of_un_answ.pkl', 'rb') as f:
    answers_2020 = pickle.load(f)
    
with open('/notebook/touche2021/baseline.qrels', 'r') as f:
    qrels_lines = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
qrels = [x.strip().split() for x in qrels_lines] 


Create qrels dict:

In [7]:
qrels[:3]

[['1',
  'Q0',
  'clueweb12-1214wb-88-29751',
  '1',
  '2406.7341',
  'MyBaselineFilterResponse'],
 ['1',
  'Q0',
  'clueweb12-1811wb-62-08418',
  '2',
  '2396.6697',
  'MyBaselineFilterResponse'],
 ['1',
  'Q0',
  'clueweb12-0200wb-79-18105',
  '3',
  '2270.9827',
  'MyBaselineFilterResponse']]

In [21]:
with open('touche2020-task2-relevance-withbaseline.qrels', 'r') as f:
        qrels_lines = f.readlines()

qrels = [x.strip().split() for x in qrels_lines] 

qrels_dict = {}
for elem in qrels:
    query, noninf, docno, rank = elem
    if (query in qrels_dict.keys()):
        qrels_dict[query].append((docno, rank))
    else:
        qrels_dict[query] = []
        qrels_dict[query].append((docno, rank))

In [9]:
qrels_dict['46'][:13]

[('clueweb12-1002wb-27-05922', '1644.429', '1'),
 ('clueweb12-0817wb-98-06373', '1067.8562', '2'),
 ('clueweb12-0300wb-78-19020', '1048.0087', '3'),
 ('clueweb12-0101wb-78-30258', '1017.8835', '4'),
 ('clueweb12-0900tw-04-12128', '992.8103', '5'),
 ('clueweb12-0814wb-83-33048', '990.42786', '6'),
 ('clueweb12-0205wb-24-11871', '953.9014', '7'),
 ('clueweb12-1713wb-94-12356', '878.3204', '8'),
 ('clueweb12-0407wb-81-05758', '877.8946', '9'),
 ('clueweb12-1309wb-69-27782', '857.4074', '10'),
 ('clueweb12-0201wb-99-15565', '719.1944', '11'),
 ('clueweb12-0912wb-55-18484', '689.3756', '12'),
 ('clueweb12-1012wb-04-07786', '612.3836', '13')]

In [10]:
answers_2020['46'][0]

(1644.429,
 'clueweb12-1002wb-27-05922',
 'pca beverage 101 - beer',
 'the finest ciders in north america may well emanate from the province of quebec in canada, which has an established artisinal cider industry with historical ties to normandy cider. these quebecois ciders are slowly starting to appear on the us market. herb-spiced and fruit beers.')

In [11]:
def clean_punct(s):
    s = re.sub(r'[^\w\s]','',s)
    return s

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()


In [12]:
qrels[:3]

[['1',
  'Q0',
  'clueweb12-1214wb-88-29751',
  '1',
  '2406.7341',
  'MyBaselineFilterResponse'],
 ['1',
  'Q0',
  'clueweb12-1811wb-62-08418',
  '2',
  '2396.6697',
  'MyBaselineFilterResponse'],
 ['1',
  'Q0',
  'clueweb12-0200wb-79-18105',
  '3',
  '2270.9827',
  'MyBaselineFilterResponse']]

In [13]:
for elem in qrels:
    if (elem[2] == 'clueweb12-1214wb-88-29751'):
        print (elem)

['1', 'Q0', 'clueweb12-1214wb-88-29751', '1', '2406.7341', 'MyBaselineFilterResponse']


In [14]:
qrels_df_test

NameError: name 'qrels_df_test' is not defined

Create proper pandas df documents and qrels

In [33]:
import pandas as pd
import re

info_df = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"],dtype=object)
info_df_test = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"], dtype=object)
qrels_df = pd.DataFrame(columns=["qid", "docno", "score", "rank"],dtype=object)
qrels_df_test = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)

for elem in topics_2020:
        qid, query = elem[0], elem[1].strip('\n')
        if (int(qid) >= 40):
            for qrel in qrels_dict[qid]:
                docno, label = qrel
                df_row = {"qid":qid, "docno":docno, "label":label}
                qrels_df_test = qrels_df_test.append(df_row, ignore_index= True)

In [31]:
with open('touche2020-task2-relevance-withbaseline.qrels', 'r') as f:
        qrels_lines = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
qrels = [x.strip().split() for x in qrels_lines] 

qrels_dict = {}
for elem in qrels:
    query, noninf, docno, label = elem
    if (query in qrels_dict.keys()):
        qrels_dict[query].append((docno, label))
    else:
        qrels_dict[query] = []
        qrels_dict[query].append((docno, label))

In [35]:
qrels_df_test.to_pickle("qrels_df_test_supernew")

In [34]:
qrels_df_test.head()

,qid,docno,label
0,40,clueweb12-1608wb-39-15329,0
1,40,clueweb12-1700tw-30-03487,0
2,40,clueweb12-0811wb-62-19220,0
3,40,clueweb12-1200tw-76-05864,0
4,40,clueweb12-1414wb-85-24273,0


In [14]:
import numpy as np
result = create_featured_dataset(info_df)

0
1
2
3
4
5
6
7
8


In [8]:
import pandas as pd

info_df = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"],dtype=object)
info_df_test = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"], dtype=object)
qrels_df = pd.DataFrame(columns=["qid", "docno", "score", "rank"],dtype=object)
qrels_df_test = pd.DataFrame(columns=["qid", "docno", "score", "rank"],dtype=object)


def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()

for elem in topics_2020:
    qid, query = elem[0], elem[1].strip('\n')
    query = re.sub(r'[^\w\s]','',query)
    query = cleanhtml(query)
    my_extractor.from_string(query)
    structures = my_extractor.get_params()

    for ind, answer in enumerate(answers_2020[qid]):
        docno = answer[1]
        score = answer[0]
        text = answer[3]
        if (int(qid) >= 40):
            nlu_score = count_score(text, structures)
            objs_score = count_score_obj(text, structures)
            ap_score = count_score_asp_pred(text, structures)
            is_retrieved = count_score_nlu(structures)
            df_row = {"qid":qid, "query":query, "docno":docno, "text":text, "baseline_scores":score, "is_retrieved":is_retrieved, "ap_score":ap_score, "objs_score":objs_score}
            info_df = info_df.append(df_row, ignore_index= True)

in extractor get params 0
self prredicates  
extract_objects_predicates tags ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
extract_objects_predicates words ['what', 'is', 'the', 'difference', 'between', 'sex', 'and', 'love']
[]
[]
[]
len(objects) 0
in extractor get params 0
self prredicates  
extract_objects_predicates tags ['O', 'O', 'B-Predicate', 'O', 'B-Object', 'O', 'O', 'B-Object']
extract_objects_predicates words ['which', 'is', 'better', 'a', 'laptop', 'or', 'a', 'desktop']
2
ind, word 2 better
old start, starts 9 set()
string   a laptop or a desktop
['laptop', 'desktop']
['better']
[]
len(objects) 2
in extractor get params 0
self prredicates  
extract_objects_predicates tags ['O', 'O', 'B-Predicate', 'B-Object', 'O', 'B-Object']
extract_objects_predicates words ['which', 'is', 'better', 'canon', 'or', 'nikon']
2
ind, word 2 better
old start, starts 9 set()
string   canon or nikon
['canon', 'nikon']
['better']
[]
len(objects) 2
in extractor get params 0
self prredicates  
extract_ob

In [ ]:
import pandas as pd
import re

info_df = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"],dtype=object)
info_df_test = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"], dtype=object)
qrels_df = pd.DataFrame(columns=["qid", "docno", "score", "rank"],dtype=object)
qrels_df_test = pd.DataFrame(columns=["qid", "docno", "score", "rank"],dtype=object)

for elem in topics_2020:
        qid, query = elem[0], elem[1].strip('\n')
        print (qid, query)
        query = re.sub(r'[^\w\s]','',query)
        query = cleanhtml(query)

        for ind, answer in enumerate(answers_2020[qid]):
            docno = answer[1]
            score = answer[0]
            text = answer[3]
            
            for qrel in qrels_dict[qid]:
                docno, score, rank = qrel

                df_row = {"qid":qid, "docno":docno, "score":score, "rank":rank}
            
            
            qrels_df = qrels_df.append(df_row, ignore_index= True)
            if (int(qid) >= 40):
                qrels_df_test = qrels_df_test.append(df_row, ignore_index= True)

        print (qrels_df_test.head(15))
            
#textscorerTf = pt.text.scorer(body_attr="text", wmodel="Tf")
#rtr = textscorerTf.transform(

In [46]:
print(len(info_df),len(qrels_df_test))

0 8818


In [ ]:
qrels_df_test['rank']

### Simple ranking

In [ ]:
from pyterrier import text

In [ ]:
from pyterrier.text import scorer

In [9]:
def create_featured_dataset(some_df):
    from pyterrier import text
    from pyterrier.text import scorer
    print (0)
    textscorerTf = text.scorer(body_attr="text", wmodel='BM25', sort=False)
    rtr_bm = textscorerTf.transform(some_df)
    print (1)
    textscorerTf = text.scorer(body_attr="text", wmodel='Tf')
    rtr_tf = textscorerTf.transform(some_df)
    print (2)
    textscorerTf = text.scorer(body_attr="text", wmodel='PL2')
    rtr_pl2 = textscorerTf.transform(some_df)
    print (3)
    textscorerTf = text.scorer(body_attr="text", wmodel='DFIC')
    rtr_dfic = textscorerTf.transform(some_df)
    print (4)
    rtr_pl2_for_merge = rtr_pl2[['qid', 'docno', 'score']]
    rtr_pl2_for_merge = rtr_pl2_for_merge.rename(columns={"score": "score_pl2"})
    print (5)
    rtr_tf_for_merge = rtr_tf[['qid', 'docno', 'score']]
    rtr_tf_for_merge = rtr_tf_for_merge.rename(columns={"score": "score_tf"})
    print (6)
    rtr_bm_for_merge = rtr_bm[['qid', 'docno', 'score']]
    rtr_bm_for_merge = rtr_bm_for_merge.rename(columns={"score": "score_bm"})
    print (7)
    rtr_dfic_for_merge = rtr_dfic[['qid', 'docno', 'score']]
    rtr_dfic_for_merge = rtr_dfic_for_merge.rename(columns={"score": "score_dfic"})
    print (8)
    result = pd.merge(rtr_pl2_for_merge, rtr_tf_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_bm_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_dfic_for_merge, on=["qid", "docno"])
    result = pd.merge(result, some_df, on=["qid", "docno"])
    zipped = [result["score_pl2"], result["score_tf"], result["score_bm"], result["score_dfic"], result['baseline_scores'], result["is_retrieved"], result["ap_score"], result["objs_score"]]
    unzipped_object = zip(*zipped)
    unzipped_list = list(unzipped_object)
    list_of_features = [np.array(elem) for elem in unzipped_list]
    result['features'] = list_of_features
    return result



In [ ]:
type(info_df['qid'][9])

In [7]:
result = create_featured_dataset(info_df)

NameError: name 'create_featured_dataset' is not defined

In [154]:
test_ds = create_featured_dataset(info_df_test)

0
1
2
3
4
5
6
7
8


In [290]:
qrels_df_test.head()

,qid,docno,score,rank
0,40,clueweb12-0716wb-20-08246,2777.8997,1
1,40,clueweb12-0805wb-58-15744,2604.5032,2
2,40,clueweb12-0604wb-65-25567,2586.6353,3
3,40,clueweb12-0700tw-41-20357,2535.3645,4
4,40,clueweb12-0700tw-61-08228,2500.514,5


In [43]:
test_ds.head()

NameError: name 'test_ds' is not defined

In [287]:
test_ds.to_pickle("test_ds.pkl")
qrels_df_test.to_pickle("qrels_df_test.pkl")

In [288]:
qrels_df_test.to_pickle("qrels_df_test.pkl")

In [202]:
#qrels_df_test = pd.read_pickle("baseline.qrels")

In [254]:
qrels_df_test.head()

,qid,docno,score,rank
0,40,clueweb12-1107wb-63-03247,362.7572,1000
1,40,clueweb12-1107wb-63-03247,362.7572,1000
2,40,clueweb12-1107wb-63-03247,362.7572,1000
3,40,clueweb12-1107wb-63-03247,362.7572,1000
4,40,clueweb12-1107wb-63-03247,362.7572,1000


In [238]:
set(list(qrels_df_test['label']))

{172, 280, 366, 1000}

In [235]:
qrels_df_test["rank"] = [int(elem) for elem in qrels_df_test["rank"]]
qrels_df_test.groupby("qid", sort=False)["rank"].rank(ascending=False, method="first").astype(int)

0          1
1          2
2          3
3          4
4          5
        ... 
8813     996
8814     997
8815     998
8816     999
8817    1000
Name: label, Length: 8818, dtype: int64

In [281]:
qrels_df_test.head(15)

,qid,docno,score,rank
0,40,clueweb12-0716wb-20-08246,2777.8997,1
1,40,clueweb12-0805wb-58-15744,2604.5032,2
2,40,clueweb12-0604wb-65-25567,2586.6353,3
3,40,clueweb12-0700tw-41-20357,2535.3645,4
4,40,clueweb12-0700tw-61-08228,2500.514,5
5,40,clueweb12-0813wb-32-14298,2494.0137,6
6,40,clueweb12-0500tw-35-13168,2450.2397,7
7,40,clueweb12-0900tw-84-01537,2427.411,8
8,40,clueweb12-0508wb-13-30410,2300.9521,9
9,40,clueweb12-1100tw-55-05936,2296.274,10


In [ ]:
qrels_df_test.

In [ ]:
qrels_df_test.head()

In [172]:
set(list(qrels_df_test['label']))

{'0', '1', '2'}

In [284]:
def write_qrels(output_dir, name, rtr):
    print ("rtr in write qrels ", rtr.head())
    qids = rtr['qid']
    Q0s = [0 for elem in qids]
    docs = rtr['docno']
    ranks = rtr['rank']
    scores = rtr['score']
    tags = [name for elem in qids]
    common_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
    print ("common_list ", common_list[:3])
    with open(output_dir + name + '.qrels', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))
    print ("written " + name +'.txt')

In [286]:
write_qrels("/notebook/touche2021/output", "baseline_test", qrels_df_test)

rtr in write qrels    qid                      docno      score rank
0  40  clueweb12-0716wb-20-08246  2777.8997    1
1  40  clueweb12-0805wb-58-15744  2604.5032    2
2  40  clueweb12-0604wb-65-25567  2586.6353    3
3  40  clueweb12-0700tw-41-20357  2535.3645    4
4  40  clueweb12-0700tw-61-08228   2500.514    5
common_list  [('40', 0, 'clueweb12-0716wb-20-08246', '1', '2777.8997', 'baseline_test'), ('40', 0, 'clueweb12-0805wb-58-15744', '2', '2604.5032', 'baseline_test'), ('40', 0, 'clueweb12-0604wb-65-25567', '3', '2586.6353', 'baseline_test')]
written baseline_test.txt


In [156]:
!pwd

/notebook/touche2021


In [75]:
assert set(test_ds['qid']) == set(qrels_df_test['qid'])

## Ranking with LTR

In [154]:
#bm25 = pt.BatchRetrieve(index, wmodel="BM25")
#tf = pt.BatchRetrieve(index, wmodel="Tf")
#pl2 = pt.BatchRetrieve(index, wmodel="PL2")
#pipeline = (rtr_tf ** rtr_pl2)

In [128]:
def add_ranks1(rtr : pd.DataFrame) -> pd.DataFrame:
    """
        Canonical method for adding a rank column which is calculated based on the score attribute
        for each query. Note that the dataframe is NOT sorted by this operation.
        Arguments
            df: dataframe to create rank attribute for
    """
    rtr.drop(columns=["rank"], errors="ignore", inplace=True)
    if len(rtr) == 0:
        rtr["rank"] = pd.Series(index=rtr.index, dtype='int64')
        return rtr
    print (0)
    # -1 assures that first rank will be FIRST_RANK
    rtr["rank"] = rtr.groupby("qid", sort=False)["score"].rank(ascending=False, method="first").astype(int) -1 + 1
    print (1)
    if True:
        rtr.sort_values(["qid", "rank"], ascending=[True,True], inplace=True)
    return rtr

def transform(model, test_DF):
    """
    Predicts the scores for the given topics.

    Args:
        topicsTest(DataFrame): A dataframe with the test topics.
    """
    test_DF = test_DF.copy()

    # check for change in number of features
    found_numf = test_DF.iloc[0].features.shape[0]
    if model.num_f is not None:
        if found_numf != model.num_f:
            raise ValueError("Expected %d features, but found %d features" % (model.num_f, found_numf))
    if hasattr(model.learner, 'feature_importances_'):
        if len(model.learner.feature_importances_) != found_numf:
            raise ValueError("Expected %d features, but found %d features" % (len(model.learner.feature_importances_), found_numf))

    test_DF["score"] = model.learner.predict(np.stack(test_DF["features"].values))
    test_DF["qid"] = [int(elem) for elem in test_DF["qid"]]
    return add_ranks1(test_DF)

In [291]:
qrels_df_test

,qid,docno,score,rank
0,40,clueweb12-0716wb-20-08246,2777.8997,1
1,40,clueweb12-0805wb-58-15744,2604.5032,2
2,40,clueweb12-0604wb-65-25567,2586.6353,3
3,40,clueweb12-0700tw-41-20357,2535.3645,4
4,40,clueweb12-0700tw-61-08228,2500.514,5
...,...,...,...,...
8813,50,clueweb12-1018wb-85-08785,22.560774,996
8814,50,clueweb12-1018wb-40-13246,22.558096,997
8815,50,clueweb12-1511wb-07-28336,22.408552,998
8816,50,clueweb12-0312wb-31-18414,21.938938,999


In [248]:
qrels_df_test.head()

,qid,docno,label,rank
0,40,clueweb12-1107wb-63-03247,NaN,1000
1,40,clueweb12-1107wb-63-03247,NaN,1000
2,40,clueweb12-1107wb-63-03247,NaN,1000
3,40,clueweb12-1107wb-63-03247,NaN,1000
4,40,clueweb12-1107wb-63-03247,NaN,1000


In [129]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10)
rf_pipe = pt.ltr.apply_learned_model(rf)
rf_pipe.fit(result, qrels_df)
answs = transform(rf_pipe, test_ds)
#pt.Experiment([result, rf_pipe], result, qrels_df, ["map"], names=["BM25 Baseline", "LTR"])

0
1


In [219]:
def write_qrels(name, rtr):
    qids = rtr['qid']
    Q0s = [0 for elem in qids]
    docs = rtr['docno']
    scores = rtr['label']
    ranks = rtr['rank']
    tags = [name for elem in qids]
    common_list = list(zip(qids, Q0s, docs, ranks, tags))
    with open(name +'.qrels', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))
    print ("written " + name +'_test.qrels')

In [192]:
rtr["rank"] = rtr.groupby("qid", sort=False)["label"].rank(ascending=False).astype(int) -1 + 1

rtr.sort_values(["qid", "rank"], ascending=[True,True], inplace=True)

NameError: name 'rtr' is not defined

In [220]:
qrels_df_test["label"] = [int(elem) for elem in qrels_df_test["label"]]
qrels_df_test["rank"] = qrels_df_test.groupby("qid", sort=False)["label"].rank(ascending=False, method="first").astype(int) -1 + 1


qrels_df_test.sort_values(["qid", "rank"], ascending=[True,True], inplace=True)

In [ ]:
write_qrels("baseline_test", qrels_df_test)

In [218]:
qrels_df_test[1200:1250]

,qid,docno,label,rank
1200,41,clueweb12-1311wb-44-25723,2,201
1201,41,clueweb12-1311wb-44-25723,2,202
1202,41,clueweb12-1311wb-44-25723,2,203
1203,41,clueweb12-1311wb-44-25723,2,204
1204,41,clueweb12-1311wb-44-25723,2,205
1205,41,clueweb12-1311wb-44-25723,2,206
1206,41,clueweb12-1311wb-44-25723,2,207
1207,41,clueweb12-1311wb-44-25723,2,208
1208,41,clueweb12-1311wb-44-25723,2,209
1209,41,clueweb12-1311wb-44-25723,2,210


In [ ]:
rtr["rank"] = rtr.groupby("qid", sort=False)["score"].rank(ascending=False, method="first").astype(int) -1 + 1
    print (1)
    if True:
        rtr.sort_values(["qid", "rank"], ascending=[True,True], inplace=True)

In [187]:
qrels_df_test.groupby("qid", sort=False)["label"].head()

0       1
1       1
2       1
3       1
4       1
1000    2
1001    2
1002    2
1003    2
1004    2
2000    0
2001    0
2002    0
2003    0
2004    0
3000    0
3001    0
3002    0
3003    0
3004    0
3172    1
3173    1
3174    1
3175    1
3176    1
4172    1
4173    1
4174    1
4175    1
4176    1
4538    0
4539    0
4540    0
4541    0
4542    0
4818    0
4819    0
4820    0
4821    0
4822    0
5818    1
5819    1
5820    1
5821    1
5822    1
6818    0
6819    0
6820    0
6821    0
6822    0
7818    0
7819    0
7820    0
7821    0
7822    0
Name: label, dtype: object

In [51]:
write_qrels("RF", answs)

written RF_test.qrels


In [39]:
rtr = qrels_df_test
qids = rtr['qid']
Q0s = [0 for elem in qids]
docs = rtr['docno']
ranks = rtr['label']

common_list = list(zip(qids, Q0s, docs, ranks))
with open("standart_test" +'.qrels', 'w') as fp:
    fp.write('\n'.join('%s %s %s %s' % x for x in common_list))
print ("written " + 'standart' +'_test.qrels')

written standart_test.qrels


In [34]:
qrels_df_test[:3]

,qid,docno,label
0,40,clueweb12-1608wb-39-15329,0
1,40,clueweb12-1700tw-30-03487,0
2,40,clueweb12-0811wb-62-19220,0


In [67]:
#write_qrels("standart", qrels_df_test)

In [77]:
result.head()

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,label,features
0,1,clueweb12-0200wb-79-18105,11.626052,9.0,23.103274,21.680909,what is the difference between sex and love,"home &gt; articles &gt; sex, sexuality &amp; p...",2270.98270,0.0,0.0,0.0,[2],"[11.62605208595209, 9.0, 23.103273698299724, 2..."
1,1,clueweb12-1214wb-88-29751,11.339902,10.0,22.708421,21.520754,what is the difference between sex and love,sex may or may not include penetration. differ...,2406.73410,0.0,0.0,0.0,[2],"[11.339901919789828, 10.0, 22.708421284786994,..."
2,1,clueweb12-0001wb-05-12311,8.889443,5.0,19.074886,17.001041,what is the difference between sex and love,her mantra &quot;no more bad dates&quot; is a ...,872.68384,0.0,0.0,0.0,[0],"[8.889443285070843, 5.0, 19.074885717934308, 1..."
3,1,clueweb12-1400tw-53-02910,7.796204,4.0,17.229402,15.395087,what is the difference between sex and love,"leave a comment in the past, i’ve had sex, i’v...",1970.16040,0.0,0.0,0.0,[2],"[7.796203967373771, 4.0, 17.22940189701382, 15..."
4,1,clueweb12-1801wb-11-09209,7.637733,4.0,14.292758,13.839917,what is the difference between sex and love,nor can we make love without god for he is the...,1178.09250,0.0,0.0,0.0,[2],"[7.637732581490534, 4.0, 14.292757865862576, 1..."


## LightGMB

In [94]:
type(result["label"].values[4])

numpy.int64

In [99]:
cord19 = pt.datasets.get_dataset('antique')

In [100]:
topics = cord19.get_topics(variant='train')
qrels = cord19.get_qrels(variant='train')

In [101]:
import os
pt_index_path = './terrier_cord19_blocks'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)

In [103]:
from sklearn.model_selection import train_test_split

train_topics, test_topics = train_test_split(topics, test_size=15, random_state=42)
train_qrels, test_qrels =  train_test_split(qrels, test_size=15, random_state=42)


In [105]:
train_topics.head()

,qid,query
2030,3638873,how to get rid of fat from beer
949,3873558,how to get good tax returns
1820,4116521,how can i get rid of permanent marker
1684,1054142,so how bout them apples
1703,1243448,how do you get yahoo messenger


In [110]:
result['label']

0       2
1       2
2       0
3       2
4       2
       ..
1048    0
1049    0
1050    0
1051    0
1052    0
Name: label, Length: 1053, dtype: int64

In [1]:
import lightgbm as lgb

# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

rf_pipe = pt.ltr.apply_learned_model(lmart_l, form="ltr")

#rf_pipe.learner.fit(np.stack(train_DF["features"].values), train_DF["label"].values)
rf_pipe.fit(result, qrels_df, test_ds, qrels_df_test)
#answs = transform(rf_pipe, test_ds)



NameError: name 'pt' is not defined

In [116]:
result.head()

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,label,features
0,1,clueweb12-0200wb-79-18105,11.626052,9.0,23.103274,21.680909,what is the difference between sex and love,"home &gt; articles &gt; sex, sexuality &amp; p...",2270.98270,0.0,0.0,0.0,2,"[11.62605208595209, 9.0, 23.103273698299724, 2..."
1,1,clueweb12-1214wb-88-29751,11.339902,10.0,22.708421,21.520754,what is the difference between sex and love,sex may or may not include penetration. differ...,2406.73410,0.0,0.0,0.0,2,"[11.339901919789828, 10.0, 22.708421284786994,..."
2,1,clueweb12-0001wb-05-12311,8.889443,5.0,19.074886,17.001041,what is the difference between sex and love,her mantra &quot;no more bad dates&quot; is a ...,872.68384,0.0,0.0,0.0,0,"[8.889443285070843, 5.0, 19.074885717934308, 1..."
3,1,clueweb12-1400tw-53-02910,7.796204,4.0,17.229402,15.395087,what is the difference between sex and love,"leave a comment in the past, i’ve had sex, i’v...",1970.16040,0.0,0.0,0.0,2,"[7.796203967373771, 4.0, 17.22940189701382, 15..."
4,1,clueweb12-1801wb-11-09209,7.637733,4.0,14.292758,13.839917,what is the difference between sex and love,nor can we make love without god for he is the...,1178.09250,0.0,0.0,0.0,2,"[7.637732581490534, 4.0, 14.292757865862576, 1..."


In [115]:
train_DF.head()

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,label_x,features,label_y
0,1,clueweb12-0200wb-79-18105,11.626052,9.0,23.103274,21.680909,what is the difference between sex and love,"home &gt; articles &gt; sex, sexuality &amp; p...",2270.98270,0.0,0.0,0.0,2,"[11.62605208595209, 9.0, 23.103273698299724, 2...",0
1,1,clueweb12-1214wb-88-29751,11.339902,10.0,22.708421,21.520754,what is the difference between sex and love,sex may or may not include penetration. differ...,2406.73410,0.0,0.0,0.0,2,"[11.339901919789828, 10.0, 22.708421284786994,...",0
2,1,clueweb12-0001wb-05-12311,8.889443,5.0,19.074886,17.001041,what is the difference between sex and love,her mantra &quot;no more bad dates&quot; is a ...,872.68384,0.0,0.0,0.0,0,"[8.889443285070843, 5.0, 19.074885717934308, 1...",0
3,1,clueweb12-1400tw-53-02910,7.796204,4.0,17.229402,15.395087,what is the difference between sex and love,"leave a comment in the past, i’ve had sex, i’v...",1970.16040,0.0,0.0,0.0,2,"[7.796203967373771, 4.0, 17.22940189701382, 15...",0
4,1,clueweb12-1801wb-11-09209,7.637733,4.0,14.292758,13.839917,what is the difference between sex and love,nor can we make love without god for he is the...,1178.09250,0.0,0.0,0.0,2,"[7.637732581490534, 4.0, 14.292757865862576, 1...",0


In [107]:
test_ds.head()

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,label,features
0,40,clueweb12-1700tw-30-03487,6.597378,6.5,13.481996,13.584072,which operating system has better performance ...,windows 7 professional and ultimate users will...,2255.4893,1.0,0.0,2.0,0,"[6.597377968911245, 6.5, 13.481995561686743, 1..."
1,40,clueweb12-0300tw-05-10074,6.555987,6.5,13.607904,13.723079,which operating system has better performance ...,windows 8 is a coin with two very different si...,1917.8842,1.0,0.0,2.0,1,"[6.555986727741695, 6.5, 13.60790372983337, 13..."
2,40,clueweb12-1100tw-64-01206,5.987314,3.0,13.224987,11.186697,which operating system has better performance ...,it targeted even better performance for window...,1942.9227,1.0,1.0,2.0,2,"[5.987314476428924, 3.0, 13.224986571587602, 1..."
3,40,clueweb12-0700tw-41-20357,5.911963,8.0,11.903069,12.959875,which operating system has better performance ...,microsoft&#x27;s embedded windows operating sy...,2535.3645,1.0,0.0,2.0,0,"[5.911962693211354, 8.0, 11.903069461047062, 1..."
4,40,clueweb12-1700wb-29-15033,5.806860,8.0,12.168118,12.562240,which operating system has better performance ...,windows 8 improves on work on minwin which sta...,1950.5464,1.0,0.0,2.0,2,"[5.80685983863876, 8.0, 12.168117999890416, 12..."
